<a href="https://colab.research.google.com/github/hsynj/AI-Special-HW/blob/main/HW1-Movie-RAG-Recommender/01_Movie_RAG_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple RAG Exercise (Class Assignment - Solution)

This notebook contains the complete solution to the 'Simple RAG Exercise' for the 'Special Topics in AI' course.

The goal of this exercise is to build a basic **RAG (Retrieval-Augmented Generation)** pipeline to recommend movies based on a user's query.

### Key Features Implemented:
* **Data Loading:** Loading the TMDB 5,000 movie dataset from Hugging Face `datasets`.
* **Embedding:** Using `sentence-transformers` to encode movie overviews into vectors.
* **Indexing:** Storing the vectors and movie metadata in a `Qdrant` in-memory vector database.
* **Hybrid Search:** Performing a query that combines:
    1.  **Semantic Search** (finding similar overviews).
    2.  **Metadata Filtering** (filtering by `release_date` for the 1990s).
* **Generation:** Using an LLM (Gemini/GPT-4) to generate a user-friendly response based on the retrieved context.

In [ ]:
# requirements
!pip install datasets qdrant-client sentence-transformers google-generativeai openai rich

In [ ]:
from rich.console import Console

# Create a console (we will use the default theme)
console = Console()

In [ ]:
from datasets import load_dataset

dataset = load_dataset('AiresPucrs/tmdb-5000-movies')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
console.print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 
'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 
'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew'],
        num_rows: 4803
    })
})

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

# Create the embedding encoder
encoder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
console.print(encoder)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True,
'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': 
False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# Create collection to store the wine rating data
collection_name="movies"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipython-input-2094978642.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [ ]:
points_list = []
for i, movie in enumerate(dataset['train']):
  if movie['overview']:
    print(f"{i+1} - Processing ID: {movie['id']}, Title: {movie['original_title']}")
    points_list.append(
        models.PointStruct(
            id=movie['id'],
            vector=encoder.encode(movie['overview']).tolist(),
            payload=movie
        )
    )

print("\nStarting upload to Qdrant...")
qdrant.upload_points(
    collection_name=collection_name,
    points=points_list
)
print("Upload complete!")

In [67]:
console.print(
    qdrant
    .get_collection(
        collection_name=collection_name
    )
)

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    vectors_count=None,
    indexed_vectors_count=0,
    points_count=4800,
    segments_count=1,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=768,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None,
                datatype=None,
                multivector_config=None
            ),
            shard_number=None,
            sharding_method=None,
            replication_factor=None,
            write_consistency_factor=None,
            read_fan_out_factor=None,
            on_disk_payload=None,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=None,
            payload_m=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=20000,
            flush_interval_sec=5,
            max_optimization_threads=1
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0),
        quantization_config=None,
        strict_mode_config=None
    ),
    payload_schema={}
)

In [68]:
user_prompt = "Love story between an Asian king and European teacher"

In [69]:
query_vector = encoder.encode(user_prompt).tolist()

In [70]:
from qdrant_client import models

query_filter= models.Filter(
  must=[
      models.FieldCondition(
          key='release_date',
          range=models.DatetimeRange(
              gte='1990-01-01T00:00:00Z',
              lte='1999-12-31T23:59:59Z'
          )
      )
  ]
)

In [71]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1,
    query_filter=query_filter,
)

/tmp/ipython-input-3678443016.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [72]:
from rich.text import Text
from rich.table import Table

table = Table(title="Retrieval Results", show_lines=True)

table.add_column("ID", style="#e0e0e0")
table.add_column("Original Title", style="#e0e0e0")
table.add_column("Overview", style="bright_red")
table.add_column("Score", style="#89ddff")

for hit in hits:
    table.add_row(
        str(hit.payload["id"]),
        hit.payload["original_title"],
        f'{hit.payload["overview"]}',
        f"{hit.score:.4f}"
    )

console.print(table)

                                                 Retrieval Results                                                 
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ ID   ┃ Original Title    ┃ Overview                                                                    ┃ Score  ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 1439 │ Anna and the King │ The story of the romance between the King of Siam (now Thailand) and the    │ 0.6419 │
│      │                   │ widowed British school teacher Anna Leonowens during the 1860's. Anna       │        │
│      │                   │ teaches the children and becomes romanced by the King. She convinces him    │        │
│      │                   │ that a man can be loved by just one woman.                                  │        │
└──────┴───────────────────┴─────────────────────────────────────────────────────────────────────────────┴────────┘

In [73]:
# define a variable to hold the search results with specific fields
search_results = [
    {
        'original_title': hit.payload['original_title'],
        'title': hit.payload['title'],
        'overview': hit.payload['overview'],
        'release_date': hit.payload['release_date'],
        'popularity': hit.payload['popularity']
    } for hit in hits]

In [74]:
console.print(search_results)

[
    {
        'original_title': 'Anna and the King',
        'title': 'Anna and the King',
        'overview': "The story of the romance between the King of Siam (now Thailand) and the widowed British 
school teacher Anna Leonowens during the 1860's. Anna teaches the children and becomes romanced by the King. She 
convinces him that a man can be loved by just one woman.",
        'release_date': '1999-12-16',
        'popularity': 13.327372
    }
]

In [75]:
import google.generativeai as genai
from google.colab import userdata
from rich.panel import Panel
from rich.text import Text

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    console.print(f"[bold red]Error:[/bold red] Could not configure Gemini. Make sure you have stored 'GOOGLE_API_KEY' in Colab Secrets.")
    raise e

system_prompt = "You are a helpful movie recommendation assistant. You will recommend a movie based *only* on the context I provide you."
user_request = f"The user is asking for: '{user_prompt}'. Based on this, I found the following movie context: {search_results}. Please provide a recommendation for the user based *only* on this context."

full_prompt = f"{system_prompt}\n\n{user_request}"

model = genai.GenerativeModel('gemini-2.5-flash')
completion = model.generate_content(full_prompt)

response_text = Text(completion.text)
styled_panel = Panel(
    response_text,
    title="Movie Recommendation with Retrieval",
    expand=False,
    border_style="bright_yellow",
    padding=(1, 1)
)

console.print(styled_panel)

╭────────────────────────────────────── Movie Recommendation with Retrieval ──────────────────────────────────────╮
│                                                                                                                 │
│ Based on the context provided, I recommend:                                                                     │
│                                                                                                                 │
│ **Anna and the King**                                                                                           │
│                                                                                                                 │
│ This film tells the story of the romance between the King of Siam (now Thailand) and the widowed British school │
│ teacher Anna Leonowens during the 1860s.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯